<a href="https://colab.research.google.com/github/AllanBottino/Series_Temporais_Python/blob/main/Downsampling_e_Upsampling_S%C3%A9ries_Temporais_Python_Parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downsampling e Upsampling

Nos artigos anteriores sobre séries temporais lidamos com o com o resample vs asfreq, nesse vamos falar de Downsampling e Upsampling, creio que é totalmente pertinente falar disso agora, mesmo que seja de maneira simples, pois irá preparar o leitor (eu também) quando mais para frente for lidar com modelos de previsão e ter que lidar com dados desbalanceados.
Já aviso que existem melhores maneiras de se ensinar e mostrar isso (com muitos gráficos e muitos pontos coloridos), mas como já disse isso é uma explicação básica e simplista, que tem intenção de preparar o leitor para o futuro, tendo todas essas ressalvas em vista, comecemos: 

Upsampling e Downsampling são um DOS métodos para lidar com dados desequilibrados.

-Downsampling e upsampling
Já que estamos lidando com reamostragem, acho totalmente pertinente já encaixar o Downsample (subamostragem) e o Upsample (sobreamostragem) aqui. 

Upsampling e Downsampling são um dos métodos para lidar com dados desequilibrados.

-O que são dados desequilibrados?

Os dados estão desequilibrados quando uma classe de dados específica está presente em um número significativamente maior do que outra classe. Ou seja, um lado é majoritário em dados e o outro minoritário, um com muito e outro com pouco.

-Por que isso é um problema?

Os dados desequilibrados são um problema, pois o algoritmo ML é tendencioso para a classe Majoritária. Ex: digamos que em um banco a maioria dos clientes é de uma nacionalidade específica e um número menor de outra nacionalidade, portanto, se o modelo for treinado com esses dados, é mais provável que o modelo rejeite o empréstimo para a nacionalidade minoritária.

-O que devemos fazer sobre isso?

Explicação oficial:
Upsampling (Sobreamostragem) - Duplicando amostras da classe minoritária
Undersampling (Subamostragem) - Excluindo amostras da classe majoritária.

Explicação minha:
Upsampling (Sobreamostragem) – Aumentamos a frequência que os dados aparecem.
Exemplo: Podemos ver muito mais dados de minuto em minuto do que mês a mês.
Undersampling (Subamostragem) – Diminuímos a frequência que os dados aparecem.
Exemplo: O oposto do exemplo acima.

Em outras palavras, tanto a sobreamostragem quanto a subamostragem envolvem a introdução de um viés para selecionar mais amostras de uma classe do que de outra, para compensar um desequilíbrio que já está presente nos dados ou que provavelmente se desenvolverá se uma amostra puramente aleatória for coletada (Fonte : Wikipedia ).

Já me adianto dizendo que existem formas melhores de mostrar e falar de balanceamento de dados e reamostragem, com mais figuras e etc, mas do jeito que eu fiz creio que ficou bom, continuem acompanhando.
-Como os dados deveriam estar?

Se houvessem duas classes (homem ou mulher por exemplo), os dados EQUILIBRADOS significariam estar em 50% de pontos para cada uma das classes.

Para a maioria das técnicas de aprendizado de máquina, pouco desequilíbrio não é um problema. Portanto, se houver 60% de pontos para uma classe e 40% para a outra classe, isso não deve causar nenhuma degradação significativa do desempenho. Somente quando o desequilíbrio da classe é alto, por exemplo, 90% de pontos para uma classe e 10% para a outra, os critérios de otimização padrão ou medidas de desempenho podem não ser tão eficazes e precisam de modificação.

Como podem ver, não existe “Tem que ser assim”, o que existe é quanto mais equilibrados os dados estiverem melhores serão os resultados.


Agora vamos alguns exemplos práticos:

Vou carregar esse conjunto de dados:

http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

O conjunto que eu normalmente uso é o "Store item demand forecasting challenge time series", eu testei e não ficou bom para o exemplo, vamos usar esse outro que ficou bem melhor.


# 1 - Importando as Bibliotecas:

In [9]:
import pandas as pd

import numpy as np

# O comando abaixo tira umas mensagens chatas.
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import warnings
warnings.filterwarnings("ignore")

# 2 - Carregando os arquivos:

In [3]:
from google.colab import files
uploaded = files.upload()

Saving datatest.txt to datatest.txt
Saving datatest2.txt to datatest2.txt


In [4]:
import io

test = pd.read_csv(io.BytesIO(uploaded['datatest.txt']))

test2 = pd.read_csv(io.BytesIO(uploaded['datatest2.txt']))

df9 = pd.concat([test, test2])

df9[ 'date' ] = pd.to_datetime(df9['date'])

df10 = df9.copy()

df10 = df10.set_index('date')

print(df10.head(21))

                     Temperature  Humidity  ...  HumidityRatio  Occupancy
date                                        ...                          
2015-02-02 14:19:00      23.7000   26.2720  ...       0.004764          1
2015-02-02 14:19:59      23.7180   26.2900  ...       0.004773          1
2015-02-02 14:21:00      23.7300   26.2300  ...       0.004765          1
2015-02-02 14:22:00      23.7225   26.1250  ...       0.004744          1
2015-02-02 14:23:00      23.7540   26.2000  ...       0.004767          1
2015-02-02 14:23:59      23.7600   26.2600  ...       0.004779          1
2015-02-02 14:25:00      23.7300   26.2900  ...       0.004776          1
2015-02-02 14:25:59      23.7540   26.2900  ...       0.004783          1
2015-02-02 14:26:59      23.7540   26.3500  ...       0.004794          1
2015-02-02 14:28:00      23.7360   26.3900  ...       0.004796          1
2015-02-02 14:29:00      23.7450   26.4450  ...       0.004809          1
2015-02-02 14:30:00      23.7000   26.

In [5]:
df10.columns

Index(['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio',
       'Occupancy'],
      dtype='object')

Como podem ver estamos lidando com dados relacionados ao clima.

# 3 - Downsample:

Diminuímos a frequência que os dados aparecem:

In [6]:
rr = df10.resample('D').mean()

print(rr.head())

            Temperature   Humidity  ...  HumidityRatio  Occupancy
date                                ...                          
2015-02-02    21.825354  24.579990  ...       0.004001   0.349398
2015-02-03    21.438301  25.889320  ...       0.004119   0.415972
2015-02-04    21.070801  24.855041  ...       0.003845   0.263975
2015-02-05          NaN        NaN  ...            NaN        NaN
2015-02-06          NaN        NaN  ...            NaN        NaN

[5 rows x 6 columns]


O exemplo acima é um exemplo exagerado, prestem atenção nos dados originais, neles podemos ver que existem registros nas horas, nos minutos e nos segundos, mas no caso eu coloquei para vermos a informação com a frequência "D" de diária, ou seja veremos as informações apenas dia a dia. Não entendeu? No exemplo abaixo você entenderá.

# 4 - Upsample:
Aumentando a frequência que os dados aparecem:

In [7]:
pp = df10.resample('3s').mean()

print(pp.head(21))

                     Temperature  Humidity  ...  HumidityRatio  Occupancy
date                                        ...                          
2015-02-02 14:19:00       23.700    26.272  ...       0.004764        1.0
2015-02-02 14:19:03          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:06          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:09          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:12          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:15          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:18          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:21          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:24          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:27          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:30          NaN       NaN  ...            NaN        NaN
2015-02-02 14:19:33          NaN      

Talvez você tenha ficado confuso (eu fiquei por um tempo), eu até deixei o head em 21, para podermos ver bem o que acontece. Eu reamostrei os dados em 3 segundos, ou seja, eu dei o comando para mostrar as informações com a frequência de 3 segundos. Agora se o leitor olhar para o dataframe original verá que ele tem ano - mês - dia - hora - minutos - segundos. Ou seja, quando eu decidi que era para mostrar as informações no intervalo de 3 segundos, ele assim fez, mas o que acontece que só temos informações registradas em determinados períodos, no caso ele vai mostrar tudo de 3 em 3 segundos até finalmente chegar onde tem informações registradas, e como não temos tantas informações registradas no intervalo de 3 segundos, acaba que se criando vários NaN, valores nulos/vazios.

# 5 - Offset:
Você também pode adicionar um deslocamento:

In [10]:
hh = df10.resample('H',loffset='10s').mean()

print(hh.head())

                     Temperature   Humidity  ...  HumidityRatio  Occupancy
date                                         ...                          
2015-02-02 14:00:10    23.657118  27.023720  ...       0.004889   1.000000
2015-02-02 15:00:10    23.293950  28.412430  ...       0.005030   1.000000
2015-02-02 16:00:10    22.773142  26.737452  ...       0.004585   1.000000
2015-02-02 17:00:10    22.534520  24.972128  ...       0.004217   0.610169
2015-02-02 18:00:10    21.993372  24.595967  ...       0.004018   0.083333

[5 rows x 6 columns]


E o que fazer com esses dados nulos? Um dos muitos métodos para lidar com isso é a Interpolação, que falarei no próximo artigo, não percam ele, pois ele é uma continuação deste.

Abraços.
